# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [1]:
# PUT YOUR IMPORTS HERE
import altair as alt
import pandas as pd
import numpy as np
legislators = pd.read_csv("data/legislators.csv")


## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [ ]:
def legislators_df():
    legislators["state"] = legislators.jurisdiction.str[-13:-11]
    return legislators

# render dataframe
legislators_df()

# NOTE: I used this SO link to help with applying string splicing on a series
# https://stackoverflow.com/questions/25789445/pandas-make-new-column-from-string-slice-of-another-column

### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [ ]:
# NOTE: I was running into "maximum row" errors, so I followed the Altair documentation to fix it with this command:
alt.data_transformers.disable_max_rows()
# NOTE: I had trouble getting my function to actually display the chart and read this SO post for help:
# https://stackoverflow.com/questions/71789741/altair-chart-not-displaying-when-evaluated-within-a-function
def states_by_gender_initial(df):
    alt.Chart(df).mark_bar().encode(
        x="state",
        y="count(gender)",
        color="gender"
    ).display()
# render chart
states_by_gender_initial(legislators_df())

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: https://blog.datawrapper.de/gendercolor/)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png.*

In [ ]:
domain = ["Female", "Male", "Other"]
range_ = ["#8624f5", "#418743", "#f2d6a1"]

def states_by_gender_improved(df):
    main_c = alt.Chart(df).mark_bar().encode(
        x="state",
        y=alt.Y("count(gender)").stack("normalize"),
        color=alt.Color("gender").scale(domain=domain, range=range_)
    )

    line_c = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='black').encode(
        y='y'
    )

    return main_c + line_c

# render chart
states_by_gender_improved(legislators_df())

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [ ]:
def party_control_raw(df):
    main_c = alt.Chart(df).mark_bar().encode(
        x="party",
        y=alt.Y("count(gender)").stack("normalize"),
        color=alt.Color("gender").scale(domain=domain, range=range_)
      )
    line_c = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='black').encode(y='y')

    return main_c + line_c

party_control_raw(legislators_df())

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [ ]:
def clean_party_df():
    # start with the DataFrame from part 1 & return transformed copy
    df = legislators_df()
    
    # Creating 'party_code' column
    conditions = [(df.party.str.contains("Democratic")), (df.party.str.contains("Republican"))]
    codes = ["D", "R"]
    df["party_code"] = np.select(conditions, codes, default="O")

    # Filtering to remove DC, NE, and PR
    clean_df = df.loc[~df["state"].isin(("dc", "ne", "pr"))]
    return clean_df

clean_party_df()
# NOTE: Used this link to figure out the syntax for "not in" with Pandas
# https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql

### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [ ]:
def party_control_by_chamber(df):
    domain = ["D", "O", "R"]
    range_ = ["blue", "green", "red"]
    
    main_c = alt.Chart(df).mark_bar().encode(
        y="state",
        x=alt.X("count(party_code)").stack("normalize"),
        color=alt.Color("party_code").scale(domain=domain, range=range_)
    )
    
    line_c = alt.Chart(df).mark_rule().encode(x='v:Q')

    final_c = alt.layer(main_c, line_c).transform_calculate(v='0.5').facet(column='type:N')
    return final_c

party_control_by_chamber(clean_party_df())

# NOTE: I was having trouble figuring out how to deal with layering on faceted charts, and consulted
# this Altair issue on Github: https://github.com/vega/altair/issues/1124

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [ ]:
def population_combined_df():
    # State/population data
    pop = pd.read_csv("data/populations.csv")

    # Counting seats -- this is very messy, I find grouping in pandas to be tricky
    # Used this link to help with hierarchical indices: https://stackoverflow.com/questions/52774232/pandas-dataframe-issue-reset-index-does-not-remove-hierarchical-index
    leg_with_state = clean_party_df()
    seat_counts = leg_with_state.groupby(['state', 'type'], as_index=False).size()
    seat_counts_p = seat_counts.pivot(index="state", columns="type")
    seat_counts_p.columns = seat_counts_p.columns.droplevel(0)

    #Combining dataframes
    combined_df = pop.join(seat_counts_p, on='state')
    return combined_df
    
population_combined_df()

# NOTE: I was having trouble with the .count() aggregator on group by, so this SO article pointed me to using .size() instead
# https://stackoverflow.com/questions/17679089/pandas-dataframe-groupby-two-columns-and-get-counts

### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [ ]:
def scatter_pop_size(df):
    upper_c = alt.Chart(df).mark_point().encode(
        alt.X("pop_2020", axis=alt.Axis(values=[0, 10000000, 20000000, 30000000, 40000000], format="M")),
        y=alt.Y("upper").title("Seats"),
        color=alt.value('purple'),
        shape=alt.value('triangle-up'))
    lower_c = alt.Chart(df).mark_point().encode(
        alt.X("pop_2020", axis=alt.Axis(values=[0, 10000000, 20000000, 30000000, 40000000], format="M")),
        y=alt.Y("lower").title("Seats"),
        color=alt.value('orange'),
        shape=alt.value('triangle-down'))

    main_c = upper_c + lower_c
    return main_c.encode(
        alt.X("pop_2020").title("2020 Population"))
scatter_pop_size(population_combined_df())

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [ ]:
def scatter_pop_size_regression():
    df = population_combined_df()

    upper_c = alt.Chart(df).mark_point().encode(
        alt.X("pop_2020", axis=alt.Axis(values=[0, 10000000, 20000000, 30000000, 40000000], format="M")),
        y=alt.Y("upper").title("Seats"),
        color=alt.value('purple'),
        shape=alt.value('triangle-up'))

    upper_reg = upper_c.transform_regression('pop_2020', 'upper').mark_line(color='purple')
    
    lower_c = alt.Chart(df).mark_point().encode(
        alt.X("pop_2020", axis=alt.Axis(values=[0, 10000000, 20000000, 30000000, 40000000], format="M")),
        y=alt.Y("lower").title("Seats"),
        color=alt.value('orange'),
        shape=alt.value('triangle-down'))

    lower_reg = lower_c.transform_regression('pop_2020', 'lower').mark_line(color='orange')

    main_c = upper_c + lower_c + upper_reg + lower_reg
    return main_c.encode(
        alt.X("pop_2020").title("2020 Population"))

scatter_pop_size_regression()

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [ ]:
def actions_df():
    actions = pd.read_csv("data/actions_il-in-mi-wi_2021-2024.csv")

    # Converting to date, and then recoverting to get rid of timestamp
    actions.date = pd.to_datetime(actions.date, format="ISO8601", utc=True)
    actions['date'] = actions['date'].dt.date
    actions.date = pd.to_datetime(actions.date, format="%Y-%m-%d")
    return actions

actions_df()

### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [ ]:
def actions_heatmap_scaled(df):
    alt.Chart(df).mark_rect().encode(
        y='state:N',
        x=alt.X('yearweek(date):T'),
        color = 'count(description):N'
    ).display()

actions_heatmap_scaled(actions_df())
# NOTE: I was running into some issues with Altair accepting data types in aggregation, and consulted this SO post:
# https://stackoverflow.com/questions/62547160/altair-data-aggregation

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [ ]:
# MODIFY THIS LINE (exclude IL)
actions_heatmap_scaled(actions_df().loc[actions_df()['state'] != "IL"])

In [ ]:
# MODIFY THIS LINE (only IL)
actions_heatmap_scaled(actions_df().loc[actions_df()['state'] == "IL"])

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [ ]:
def actions_cumulative(df):
    alt.Chart(df).mark_line().encode(
        x= alt.X('date'),
        y = 'cumulative_count:Q',
        color='state'
    ).transform_window(
        cumulative_count='count(session)',
        sort=[{'field': 'date'}],
        groupby=['state']
    ).display()

actions_cumulative(actions_df())